In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv, dotenv_values
load_dotenv()
ENV = dotenv_values()

In [3]:
import pandas as pd
import re
from conn import get_connection
from crud import get_orders_all_years

C:\Users\bihd00\AppData\Local\Temp\ipykernel_10208\1814486045.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
conn = get_connection(
    db_server=ENV.get('DB_SERVER'),
    db_name=ENV.get('DB_NAME')
)

In [5]:
df_orders = get_orders_all_years(conn=conn)

h:\DAB\SSBI\ssbi\src\crud.py:141: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return read_sql(sql=sql, con=conn)


In [6]:
df_orders.head(3)

,Row_ID,Order_ID,Category,Sub_Category,Segment,Country_Region,State_Province,Region,City,Postal_Code,...,Customer_Name,Product_ID,Product_Name,Quantity,Sales,Discount,Profit,Order_Date,Ship_Date,Ship_Mode
0,1,US-2020-103800,Office Supplies,Paper,Consumer,United States,Texas,Central,Houston,77095,...,Darren Powers,OFF-PA-10000174,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",2,16.448,0.2,5.5512,2020-01-03,2020-01-07,Standard Class
1,10,US-2020-167199,Office Supplies,Fasteners,Home Office,United States,Kentucky,South,Henderson,42420,...,Maria Etezadi,OFF-FA-10001883,"Alliance Super-Size Bands, Assorted Sizes",4,31.120,0.0,0.3112,2020-01-06,2020-01-10,Standard Class
2,100,US-2020-125759,Furniture,Furnishings,Home Office,United States,Nevada,West,North Las Vegas,89031,...,Nathan Mautz,FUR-FU-10002111,"Master Caster Door Stop, Large Brown",2,14.560,0.0,5.5328,2020-02-08,2020-02-09,First Class


In [7]:
df_orders.dtypes

Row_ID              int64
Order_ID           object
Category           object
Sub_Category       object
Segment            object
Country_Region     object
State_Province     object
Region             object
City               object
Postal_Code        object
Customer_ID        object
Customer_Name      object
Product_ID         object
Product_Name       object
Quantity            int64
Sales             float64
Discount          float64
Profit            float64
Order_Date         object
Ship_Date          object
Ship_Mode          object
dtype: object

In [8]:
# nothing NA
df_orders[df_orders.isna().any(axis=1)]

,Row_ID,Order_ID,Category,Sub_Category,Segment,Country_Region,State_Province,Region,City,Postal_Code,...,Customer_Name,Product_ID,Product_Name,Quantity,Sales,Discount,Profit,Order_Date,Ship_Date,Ship_Mode


In [9]:
# unique row ids
df_orders['Row_ID'].nunique() == df_orders.shape[0]

True

In [10]:
# multiple different items per order
print(df_orders['Order_ID'].nunique())
df_orders['Order_ID'].nunique() == df_orders.shape[0]

5111


False

In [11]:
df_orders['Order_ID'][0]

'US-2020-103800'

In [12]:
# all orders follow the pattern of <subsidiary>-<year>-<order_id>
df_orders['Order_ID'].str.match(r'^[A-Z]{2}-\d{4}-\d+$', False, re.IGNORECASE).unique()

array([ True])

In [13]:
df_orders['Category'].unique()

array(['Office Supplies', 'Furniture', 'Technology'], dtype=object)

In [14]:
df_orders['Sub_Category'].unique()

array(['Paper', 'Fasteners', 'Furnishings', 'Appliances', 'Art',
       'Binders', 'Accessories', 'Machines', 'Labels', 'Chairs',
       'Storage', 'Tables', 'Supplies', 'Phones', 'Envelopes',
       'Bookcases', 'Copiers'], dtype=object)

In [15]:
df_orders['Segment'].unique()

array(['Consumer', 'Home Office', 'Corporate'], dtype=object)

In [16]:
df_orders['Country_Region'].unique()

array(['United States', 'Canada'], dtype=object)

In [17]:
# some misspelled state province
unique_state_provinces = df_orders['State_Province'].sort_values(ascending=True).unique()
unique_state_provinces

array(['Alabama', 'Alberta', 'Arizona', 'Arkaansas', 'Arkanas',
       'Arkansas', 'Arkasas', 'British Columbia', 'Caifornia',
       'California', 'Californiaa', 'Califorrrnia', 'Callifornia',
       'Coloorado', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Manitoba', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Brunswick', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'Newfoundland and Labrador', 'North Carolina',
       'North Dakota', 'Nova Scotia', 'Ohiio', 'Ohio', 'Oho', 'Oklahoma',
       'Ontaario', 'Ontario', 'Oregon', 'Pennsylvania',
       'Prince Edward Island', 'Quebec', 'Quebecc', 'Rhode Island',
       'Saskatchewan', 'South Carolina', 'South Dakota', 'Tennesee',
       'Tennesse', 'Tennessee', 'Texas', 'Utah', 'Vermont', '

In [18]:
def simple_similarity(s1: str, s2: str) -> float:
    if s1 == s2:
        return 1
    common_chars = set(s1) & set(s2)
    similarity = len(common_chars) / max(len(s1), len(s2))
    return similarity


def get_missspelled(
    df: pd.DataFrame,
    column_check: str,
    column_count: str,
    treshold_similarity: float = 0.5,
    treshold_count: int = 50,
) -> pd.DataFrame:
    df_counts = df[[column_check, column_count]] \
        .rename(columns={column_count: 'count'}) \
        .groupby(column_check) \
        .count() \
        .reset_index() \
        .sort_values(by='count')

    counts_dict = df_counts.set_index(column_check)['count'].to_dict()
    counts_dict_items = list(counts_dict.items()) # (<column_to_check>, count) items

    records = []
    for i in range(len(counts_dict_items)):
        for j in range(i+1, len(counts_dict_items)-1, 1):
            sp1, sp1_count = counts_dict_items[i]
            sp2, sp2_count = counts_dict_items[j]
            sim = simple_similarity(sp1, sp2)
            if (
                sim >= treshold_similarity
                and (sp1_count <= treshold_count or sp2_count <= treshold_count)
            ):
                records.append(dict(
                    a=sp1,
                    b=sp2,
                    similarity=sim
                ))
    return pd.DataFrame.from_records(records)

In [19]:
state_province_misspelled = get_missspelled(df_orders, 'State_Province', 'Row_ID', 0.51, 20)
state_province_misspelled.head()

,a,b,similarity
0,Quebecc,Quebec,0.714286
1,Ohiio,Oho,0.600000
2,Ohiio,Ohio,0.800000
3,Ontaario,Caifornia,0.555556
4,Ontaario,Manitoba,0.625000


In [20]:
state_province_misspelled_fix_dict = {
    'Quebecc': 'Quebec',
    'Ohiio': 'Ohio',
    'Oho': 'Ohio',
    'Ontaario': 'Ontario',
    'Arkaansas': 'Arkansas',
    'Arkasas': 'Arkansas',
    'Arkanas': 'Arkansas',
    'Coloorado': 'Colorado',
    'iIllinois': 'Illinois',
    'Caifornia': 'California',
    'Callifornia': 'California',
    'Californiaa': 'California',
    'Califorrrnia': 'California',
    'Tennesee': 'Tennessee',
    'Tennesse': 'Tennessee',
    'Tennessee': 'Tennessee',
}
for k, v in state_province_misspelled_fix_dict.items():
    df_orders.loc[df_orders['State_Province'] == k, 'State_Province'] = v

In [21]:
df_orders['State_Province'].sort_values().unique()

array(['Alabama', 'Alberta', 'Arizona', 'Arkansas', 'British Columbia',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Manitoba', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Brunswick', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'Newfoundland and Labrador', 'North Carolina',
       'North Dakota', 'Nova Scotia', 'Ohio', 'Oklahoma', 'Ontario',
       'Oregon', 'Pennsylvania', 'Prince Edward Island', 'Quebec',
       'Rhode Island', 'Saskatchewan', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [22]:
df_orders['Region'].sort_values().unique()

array(['Central', 'East', 'South', 'West'], dtype=object)

In [23]:
# maybe misspelled
df_orders['City'].nunique()

542

In [24]:
# can't really assume any are misspelled, even "Normal" or "Reading" exist
city_missspelled = get_missspelled(df_orders, 'City', 'Row_ID', 0.8, 20)
city_missspelled.head(10)

,a,b,similarity
0,Normal,Norman,0.833333
1,Redding,Reading,0.857143
2,Layton,Lawton,0.833333
3,Springdale,Springfield,0.818182
4,Regina,Reading,0.857143
5,Clifton,Clinton,0.857143
6,Malden,Medina,0.833333
7,Mason,Macon,0.800000
8,Loveland,Avondale,0.875000
9,Mentor,Monroe,0.833333


In [25]:
# H1C is a valid Canadian postal code...
df_orders['Postal_Code'].unique()[:50]

array(['77095', '42420', '89031', '13601', '10009', '72701', '60089',
       '60653', '65203', '33710', '43402', '94109', '74133', '19134',
       '77036', '60543', '95051', '23320', '10024', '99301', '78207',
       '11561', '94110', '75701', '31204', '01852', '19140', '43130',
       '48227', '27834', '97030', '07011', '23464', 'H1C', '93030',
       '10035', '66062', '27511', '45014', '77070', '10801', '79762',
       '98103', '94521', '10011', '28205', '43229', '19143', '85281',
       '78415'], dtype=object)

In [26]:
# seem valid for CA
df_orders[df_orders['Postal_Code'].str.len() < 5]['Postal_Code'].unique()

array(['H1C', 'M2N', 'H1A', 'T1Y', 'T5A', 'V6G', 'A0A', 'B3H', 'E1A',
       'V6B', 'T2C', 'M3C', 'V5K', 'G1B', 'C0A', 'V6Z', 'M7A', 'M5H',
       'H1B', 'V6E', 'R0H', 'S0G', 'R3R'], dtype=object)

In [27]:
# customers buying multiple products
df_orders['Customer_ID'].nunique()

804

In [28]:
df_orders['Customer_ID'].unique()[:50]

array(['DP-13000', 'ME-17320', 'NM-18445', 'JM-16195', 'KD-16345',
       'CL-12565', 'SC-20095', 'BM-11785', 'CV-12295', 'AG-10525',
       'MR-17545', 'AG-10390', 'GA-14725', 'PR-18880', 'NP-18700',
       'VP-21760', 'RB-19465', 'SB-20290', 'TB-21400', 'SZ-20035',
       'SS-20410', 'LR-17035', 'NF-18385', 'DK-13375', 'MC-17275',
       'BM-11140', 'KT-16465', 'PP-18955', 'SP-20620', 'DP-13390',
       'TS-21340', 'LH-17020', 'TT-21070', 'KC-16255', 'GZ-14470',
       'RR-19525', 'LC-17140', 'NF-18595', 'MS-17365', 'CC-12220',
       'DL-12925', 'SF-20065', 'DA-13450', 'AR-10345', 'LW-17125',
       'LW-17215', 'TA-21385', 'BN-11470', 'AF-10885', 'PT-19090'],
      dtype=object)

In [29]:
df_orders[df_orders['Customer_ID'].str.len() != 8]['Customer_ID'].unique()

array([], dtype=object)

In [30]:
df_orders['Customer_Name'].nunique()

800

In [31]:
# might be possible that thas ok
df_orders['Customer_ID'].nunique() == df_orders['Customer_Name'].nunique()

False

In [32]:
df_customer_name_unique = df_orders.groupby('Customer_Name')['Customer_ID'].nunique().reset_index()
df_customer_name_unique[df_customer_name_unique['Customer_ID'] > 1]

,Customer_Name,Customer_ID
324,Harry Olson,5


In [33]:
# hmm... same guy
df_orders[df_orders['Customer_Name'] == 'Harry Olson'][['Row_ID', 'Order_ID', 'City', 'Customer_ID', 'Customer_Name', 'Product_Name', 'Order_Date','Ship_Date']].sort_values('Customer_ID')

,Row_ID,Order_ID,City,Customer_ID,Customer_Name,Product_Name,Order_Date,Ship_Date
1154,2037,CA-2020-143500,Charlottetown,HO-15230,Harry Olson,Wilson Jones Impact Binders,2020-12-30,2020-01-03
10193,10194,CA-2023-143500,Charlottetown,HO-15230,Harry Olson,Wilson Jones Impact Binders,2023-12-30,2023-01-03
5012,5013,CA-2022-121465,Charlottetown,HO-15231,Harry Olson,GBC ProClick Spines for 32-Hole Punch,2022-06-12,2022-06-17
7194,7195,CA-2023-130494,Charlottetown,HO-15231,Harry Olson,GBC ProClick Spines for 32-Hole Punch,2023-03-16,2023-03-21
5011,5012,CA-2022-121465,Charlottetown,HO-15232,Harry Olson,Panasonic KP-350BK Electric Pencil Sharpener w...,2022-06-12,2022-06-17
7193,7194,CA-2023-130494,Charlottetown,HO-15232,Harry Olson,Panasonic KP-350BK Electric Pencil Sharpener w...,2023-03-16,2023-03-21
5010,5011,CA-2022-121465,Charlottetown,HO-15233,Harry Olson,"Global Leather and Oak Executive Chair, Black",2022-06-12,2022-06-17
7192,7193,CA-2023-130494,Charlottetown,HO-15233,Harry Olson,"Global Leather and Oak Executive Chair, Black",2023-03-16,2023-03-21
5009,5010,CA-2022-121465,Charlottetown,HO-15234,Harry Olson,Harbour Creations Steel Folding Chair,2022-06-12,2022-06-17
7191,7192,CA-2023-130494,Charlottetown,HO-15234,Harry Olson,Harbour Creations Steel Folding Chair,2023-03-16,2023-03-21


In [34]:
# for now..
df_orders.loc[df_orders['Customer_Name'] == 'Harry Olson', 'Customer_ID'] = 'HO-15230'

In [35]:
df_customer_name_unique_2 = df_orders.groupby('Customer_Name')['Customer_ID'].nunique().reset_index()
df_customer_name_unique_2[df_customer_name_unique_2['Customer_ID'] > 1]

,Customer_Name,Customer_ID


In [36]:
df_orders['Product_ID'].nunique()

1862

In [37]:
df_orders['Product_ID'].unique()[:25]

array(['OFF-PA-10000174', 'OFF-FA-10001883', 'FUR-FU-10002111',
       'OFF-AP-10001303', 'OFF-AR-10000658', 'OFF-AR-10002445',
       'OFF-BI-10001359', 'OFF-PA-10000349', 'TEC-AC-10000844',
       'TEC-AC-10001606', 'TEC-MA-10002210', 'OFF-BI-10004099',
       'OFF-LA-10001175', 'FUR-CH-10001146', 'FUR-FU-10001967',
       'OFF-ST-10000585', 'FUR-FU-10001487', 'OFF-LA-10001934',
       'FUR-CH-10004698', 'FUR-FU-10000758', 'FUR-FU-10004270',
       'FUR-FU-10004665', 'FUR-TA-10001039', 'FUR-TA-10004154',
       'FUR-FU-10002298'], dtype=object)

In [38]:
# all products follow the pattern of <category_short>-<subcategory_short>-<product_id>
df_orders['Product_ID'].str.match(r'^[A-Z]{3}-[A-Z]{2}-\d+$', False, re.IGNORECASE).unique()

array([ True])

In [39]:
df_orders['Product_Name'].nunique()

1849

In [40]:
# idk here, it is probably not that uncommon to have 2 different products with the same name
df_product_name_unique = df_orders.groupby('Product_Name')['Product_ID'].nunique().reset_index()
df_product_name_unique[df_product_name_unique['Product_ID'] > 1]

,Product_Name,Product_ID
4,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",2
259,Avery Non-Stick Binders,2
493,DAX Wood Document Frame,2
536,Easy-staple paper,8
590,Eldon Wave Desk Accessories,2
941,KI Adjustable-Height Table,2
1216,Okidata C610n Printer,2
1248,"Peel & Seel Recycled Catalog Envelopes, Brown",2
1296,Prang Drawing Pencil Set,2
1492,Staple envelope,9


In [50]:
df_orders['Quantity'].unique()

array([ 2,  4,  1,  5,  3,  7,  6,  8,  9, 10, 14, 11, 13, 12],
      dtype=int64)

In [42]:
df_orders['Sales'].unique()

array([16.448, 31.12 , 14.56 , ..., 20.72 ,  3.024, 90.93 ])

In [49]:
df_orders['Discount'].unique()

array([0.2 , 0.  , 0.4 , 0.8 , 0.3 , 0.7 , 0.1 , 0.5 , 0.6 , 0.15, 0.32,
       0.45])

In [44]:
df_orders.columns

Index(['Row_ID', 'Order_ID', 'Category', 'Sub_Category', 'Segment',
       'Country_Region', 'State_Province', 'Region', 'City', 'Postal_Code',
       'Customer_ID', 'Customer_Name', 'Product_ID', 'Product_Name',
       'Quantity', 'Sales', 'Discount', 'Profit', 'Order_Date', 'Ship_Date',
       'Ship_Mode'],
      dtype='object')

In [45]:
df_orders['Profit'].unique()

array([ 5.5512,  0.3112,  5.5328, ..., 19.791 ,  6.475 ,  2.7279])

In [46]:
df_orders['Order_Date'].unique()

array([datetime.date(2020, 1, 3), datetime.date(2020, 1, 6),
       datetime.date(2020, 2, 8), ..., datetime.date(2023, 12, 28),
       datetime.date(2023, 12, 29), datetime.date(2023, 12, 30)],
      dtype=object)

In [47]:
df_orders['Ship_Date'].unique() # was cast to just Date since all timestamps were 00:00:00

array([datetime.date(2020, 1, 7), datetime.date(2020, 1, 10),
       datetime.date(2020, 2, 9), ..., datetime.date(2023, 12, 29),
       datetime.date(2023, 1, 1), datetime.date(2023, 1, 3)], dtype=object)

In [48]:
df_orders['Ship_Mode'].unique()

array(['Standard Class', 'First Class', 'Second Class', 'Same Day'],
      dtype=object)

In [53]:
df_orders.head(3).T

,0,1,2
Row_ID,1,10,100
Order_ID,US-2020-103800,US-2020-167199,US-2020-125759
Category,Office Supplies,Office Supplies,Furniture
Sub_Category,Paper,Fasteners,Furnishings
Segment,Consumer,Home Office,Home Office
Country_Region,United States,United States,United States
State_Province,Texas,Kentucky,Nevada
Region,Central,South,West
City,Houston,Henderson,North Las Vegas
Postal_Code,77095,42420,89031
